# Prediciting Match Outcomes to beat the bookies

## Introduction

When someone is prepared to risk money, small or large, on their favourite sports team or simply as a bit of casual fun or may it be they are trying to make money. There is only one main aim and that is to beat the bookies and generate a higher return than initally staked.  

The casual gambler might be putting on bets for their favorite sports team and may play based off emotions rather than informed decisions which more than likely leads to a number of losses.  

However the professional sports bettor will use a combination of information and experience to understand whether the odds offered by the bookies are in favour of the house or not. They do not bet based off emotions but off years of experience and research. 

This project aims to replicate the knowledge that a professional punter would use to determine whether a bet is worth risking money on, whilst every individual has their own risk appetite we hope the models built in this project will be able to predict outcomes based on statistical properties of past matches. 

This could then aid bettors in making their decisions on what outcome to bet on and potentially beat the bookies. 

In [1]:
import numpy as np